<a href="https://colab.research.google.com/github/dshpedro/treinamento-h2ia/blob/main/2_buscas_sem_informacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

## Controle

In [ ]:
import numpy as np
from collections import deque
import copy
import time

In [ ]:
class Puzzle:
    def __init__(self, state=None):
        self.zeroIndex = [1, 0]
        self.initialState = state
        self.goalState = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

    def random(self):
        rng = np.random.default_rng()
        ss = np.arange(9).reshape(3, 3)
        rng.shuffle(ss)
        rng.shuffle(ss, 1)
        self.initialState = ss

    def goalTest(self, state):
        return np.array_equal(state, self.goalState)

    def actions(self, state):
        # (l)eft (d)own (u)p (r)ight
        actions = np.array([["dr", "ldr", "ld"],
                            ["dur", "ldur", "ldu"],
                            ["ur", "lur",  "lu"]])
        zeroIndex = np.where(state == 0)
        return actions[zeroIndex][0] # string of legal actions

    def result(self, state, action):
        zeroIndex = np.where(state == 0)
        mov = {
        'l': (0, -1),  # (l)eft
        'd': (1, 0),   # (d)own
        'u': (-1, 0),  # (u)p
        'r': (0, 1)    # (r)ight
        }
        zeroIndexNdarray = np.column_stack(zeroIndex).flatten()
        swapIndex = tuple( zeroIndexNdarray+ mov[action])
        s = copy.deepcopy(state)
        s[zeroIndex], s[swapIndex] = s[swapIndex], s[zeroIndex]
        return s

In [ ]:
class Node:
    def __init__(self, state=None, parent=None):
        self.state = state
        self.parent = parent
        self.action = None
        self.child = []
        self.zeroIndex = np.where(state == 0);

In [ ]:
nodes = 1
def childNode(problem, parent, action):
    child = Node()
    child.state = problem.result(parent.state, action)
    child.parent = parent
    child.action = action
    global nodes
    nodes += 1
    return child

In [ ]:
def solution(node):
    path = [node]
    while node.parent != None:
        path.append(node.parent)
        node = node.parent
    path.reverse()
    return path

In [ ]:
def solve(problem, search, limit=1):
    if search == "bfs":
        return breadthFirstSearch(problem)
    if search == "dls":
        return depthLimitedSearch(problem, limit)
    if search == "ids":
        return iterativeDeepeningSearch(problem)

## Seletor de puzzle

In [ ]:
solvableState = np.array([[1, 8, 2], [0, 4, 3], [7, 6, 5]])
unsolvableState = np.array([[1, 8, 2], [0, 4, 3], [7, 6, 4]])
puzzle = Puzzle(unsolvableState)
#puzzle.random

## Busca em largura
Busca em grafos

In [ ]:
def breadthFirstSearch(problem):
    node = Node(problem.initialState)
    if problem.goalTest(node.state):
        return solution(node)
    frontier = deque([node])
    explored = set()
    while True:
        if not bool(frontier):
            return "failure"
        node = frontier.popleft()
        explored.add(tuple(tuple(row) for row in node.state))
        for action in problem.actions(node.state):
            child = childNode(problem, node, action)
            tupleChildState = tuple(tuple(row) for row in child.state)
            if (tupleChildState not in explored) or (tupleChildState not in frontier):
                if problem.goalTest(child.state):
                    return solution(child)
                frontier.append(child)

In [ ]:
startTime = time.time()
solvedPath = solve(puzzle, "bfs")
endTime = time.time()
elapsedTime = endTime - startTime
print("%.2fs elapsed" % (elapsedTime))
print(str(nodes) + " nodes")
for node in solvedPath:
    print(node.state)
    print()

KeyboardInterrupt: 

## Busca em Profundidade
Busca em árvore

### Busca em profundidae limitada

In [ ]:
def depthLimitedSearch(problem, limit):
    node = Node(problem.initialState)
    return recursiveDLS(node, problem, limit);

def recursiveDLS(node, problem, limit):
    if problem.goalTest(node.state):
        return solution(node)
    else:
        if limit == 0:
            return "cutoff"
        else:
            cutoffOccured = False
        for action in problem.actions(node.state):
            child = childNode(problem, node, action)
            result = recursiveDLS(child, problem, limit - 1)
            if result == "cutoff":
                cutoffOccured = True
            else:
                if result != "failure":
                    return result
        if cutoffOccured:
            return "cutoff"
        else:
            return "failure"

### Busca em profundidade iterativa

In [ ]:
def iterativeDeepeningSearch(problem):
    for depth in range(20):
        result = depthLimitedSearch(problem, depth)
        if result != "cutoff":
            return result

In [ ]:
nodes = 1
startTime = time.time()
solvedPath = solve(puzzle, "ids")
endTime = time.time()
elapsedTime = endTime - startTime
print("%.2fs elapsed" % (elapsedTime))
print(str(nodes) + " nodes")
if solvedPath != "failure":
    for node in solvedPath:
        print(node.state)
        print()

## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento



### Consumo de memória

#### Busca em largura
Crescimento exponencial com o aumento da profundidade. \

O(b^d) \
b = fator de bifurcação \
d = profundidade da solução menos profunda \


#### Busca em profundidade
Crescimento linear com o incremento da profundidade máxima \

O(b*m) \
b = fator de bifuracação \
m = profundidade máxima \

### Processamento

#### Busca em largura
Crescimento exponencial com o aumento da profundidade. \

O(b^d) \
b = fator de bifurcação \
d = profundidade da solução menos profunda \


#### Busca em profundidade
Crescimento exponencial com o aumento da profundidade. \

o(b^m) \
b = fator de bifuracação \
m = profundidade máxima \

### Comparação

Observamos que a busca em profundidade aparentemente apresenta apenas vantagens em relação a em largura, porém ela sempre tende a ir a profundidade máxima, o que pode torna-la não completa se atingirmos um ciclo. No caso do problema dos blocos deslizantes por exemplo, mesmo que uma solução fosse encontrada, é improvável que seja a com o menor número de passos. \
Para diminuir o número de passos utilizamos a busca em profundidade iterativa, que incrementa a profundidade limite da busca enquanto não for encontrada uma solução com o número de passos igual à profundidade atual, dessa forma evitamos também ciclos infinitos e a busca se torna completa.

Nota: Estes comentários se referem às versões de busca em árvore dos algoritmos mencionados.